In [1]:
import tweepy
import configparser
import pandas as pd
pd.set_option("display.max_colwidth", None)
import regex as re
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import seaborn as sns
from collections import Counter
nltk.download('stopwords')
stop=set(stopwords.words('english'))
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gregormilligan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
 # read configs
config = configparser.ConfigParser()
config.read('config.ini')

api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']

access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

# authentication
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [37]:
columns = ['user_name','tweet','created_at']
kier_df = pd.DataFrame()
kier_df

""


In [7]:
def get_recent_boris_tweets():
    boris_data=[]
    boris_keywords = ['Boris Johnson','boris johnson']
    limit = 300
    boris_tweets = tweepy.Cursor(api.search_tweets, q=boris_keywords, count=100, tweet_mode='extended').items(limit)
    columns = ['user_name','tweet']
    for tweet in boris_tweets:
        boris_data.append([tweet.user.screen_name, tweet.full_text])
    global boris_df
    boris_df = pd.DataFrame(boris_data,columns=columns)

In [43]:
def get_recent_kier_tweets():
    kier_data = []
    kier_keywords = ['Kier Starmer', 'kier starmer']
    limit = 300
    kier_tweets = tweepy.Cursor(api.search_tweets, q=kier_keywords, count=100, tweet_mode='extended').items(limit)
    columns = ['user_name','tweet','created_at']
    for tweet in kier_tweets:
        kier_data.append([tweet.user.screen_name, tweet.full_text, tweet.created_at])
    placeholder_df = pd.DataFrame(kier_data,columns=columns)

In [44]:
get_recent_kier_tweets()

In [45]:
placeholder_df

NameError: name 'placeholder_df' is not defined

In [ ]:
#cleaning text
import regex as re
def cleanTxt(text):
    df.drop_duplicates()
    text = re.sub(r'@[A-Za-z0-9]+','',text) #removes @mentions
    text = re.sub(r'#','',text)#removes #
    text = re.sub(r'RT[\s]','', text)#removes RTs
    text = re.sub('https?:\/\/\S+','',text)#removes hyperlinks  
    #text = re.sub(r'[^A-Za-z]+', '', text)
    return text
df['Tweet'] = df['Tweet'].apply(cleanTxt)
#show clean text
df

In [ ]:
from textblob import TextBlob
#Creating a function to get the subjectivity (How opinionated the text is)
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
#polarity of the tweets positive negative or neutral 
def getPolarity(text):
    return TextBlob(text).sentiment.polarity
df['Subjectivity'] = df['Tweet'].apply(getSubjectivity)
df['Polarity'] = df['Tweet'].apply(getPolarity)
df

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
#visualise tweets in a wordcloud
all_words = ' '.join([tweets for tweets in df['Tweet']])
wordcloud = WordCloud(width = 500, height = 500, random_state=21, max_font_size= 119, stopwords=STOPWORDS).generate(all_words)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

#find examples of negative sentiment and polarity and show those wcs

In [ ]:
def getAnalysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'
df['Analysis'] = df['Polarity'].apply(getAnalysis)
df

In [ ]:
sortedDF = df.sort_values(by=['Polarity'])
sortedDF.to_excel('train_example2.xlsx')
sortedDF

In [ ]:
sortedDF

In [ ]:
#get top 100 worst and do a wordcloud
#visualise tweets in a wordcloud
all_words = ' '.join([tweets for tweets in sortedDF['Tweet'][0:100]])
wordcloud = WordCloud(width = 500, height = 500, random_state=21, max_font_size= 119, stopwords=STOPWORDS).generate(all_words)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


In [ ]:
plt.figure(figsize=(8,6))
for i in range(0,df.shape[0]):
    plt.scatter(df['Polarity'][i], df['Subjectivity'][i], color="violet")
    
plt.title("Railway Sentiment Analysis")
plt.xlabel("Polarity")
plt.ylabel("Subjectivty")
plt.show()

In [ ]:
%matplotlib inline 
df['Analysis'].value_counts()
plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')

df['Analysis'].value_counts().plot(kind='bar', color=['mediumseagreen','lightsteelblue', 'indianred'])
plt.show()

In [ ]:
#creating the corpus
corpus=[]
tweets = df['Tweet'].str.split()
tweets=tweets.values.tolist()
corpus=[word for i in tweets for word in i]

from collections import defaultdict
dic=defaultdict(int)
for word in corpus:
    if word in stop:
        dic[word]+=1


In [ ]:
def plot_top_stopwords_barchart(text):
    stop=set(stopwords.words('english'))
    
    new= text.str.split()
    new=new.values.tolist()
    corpus=[word for i in new for word in i]
    from collections import defaultdict
    dic=defaultdict(int)
    for word in corpus:
        if word in stop:
            dic[word]+=1
            
    top=sorted(dic.items(), key=lambda x:x[1],reverse=True)[:10] 
    x,y=zip(*top)
    
    plt.barh(x,y)

In [ ]:
plot_top_stopwords_barchart(df['Tweet'])

In [ ]:
import nltk
from nltk.corpus import stopwords
import seaborn as sns
from collections import Counter
nltk.download('stopwords')
stop=set(stopwords.words('english'))

In [ ]:
def commonWordPlot(text):
    
    counter=Counter(text)
    most=counter.most_common()

    x, y= [], []
    for word,count in most[:40]:
        if (word not in stop):
            x.append(word)
            y.append(count)
        
    sns.barplot(x=y,y=x)
commonWordPlot(corpus)

In [ ]:
public_tweets = api.home_timeline()
columns = ['Tweet']
data = []
for tweet in public_tweets:
    data.append(tweet.text)
df = pd.DataFrame(data,columns=columns)
df    
